In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import ensemble
from sklearn.cross_validation import cross_val_score
%matplotlib inline  

In [2]:
df=pd.read_csv('data.csv')
df['home']=df['matchup'].apply(lambda x: 1 if 'vs' in x else 0)
df['game_month'] = pd.DatetimeIndex(df['game_date']).month
df['game_year'] = pd.DatetimeIndex(df['game_date']).year
df["time_remaining"] = df["minutes_remaining"] * 60 + df["seconds_remaining"]
df['last_5_sec'] = df['time_remaining'] < 5

test_df=df.loc[(df.shot_made_flag.isnull()) ,:]
train_df=df.loc[(df.shot_made_flag.notnull()) ,:]

In [3]:
feature=['action_type','game_month','game_year','loc_x', 'loc_y','combined_shot_type','minutes_remaining','time_remaining','period','last_5_sec','playoffs','season','shot_distance','shot_type','shot_zone_area','shot_zone_basic','shot_zone_range','opponent','home']


X=train_df[feature]
X_test=test_df[feature]
y=train_df['shot_made_flag']
y_test=test_df['shot_made_flag']
feature_dum=['action_type','combined_shot_type','game_month','game_year','period','season','shot_type','shot_zone_area','shot_zone_basic','shot_zone_range','opponent']
X=pd.get_dummies(X,columns=feature_dum,drop_first=False)
X_test=pd.get_dummies(X_test,columns=feature_dum,drop_first=False)

feature_new=[]
for i in list(X_test.columns.values):
    if i in list(X.columns.values):
        feature_new.append(i)
X=X[feature_new]
X_test=X_test[feature_new]

from sklearn import preprocessing
normalizer = preprocessing.Normalizer()
#X = preprocessing.MaxAbsScaler().fit_transform(X)
#X = preprocessing.StandardScaler().fit_transform(X)
#X_test = preprocessing.MaxAbsScaler().fit_transform(X_test)

In [ ]:
# Test with Random Forest
from sklearn.grid_search import GridSearchCV
n_estimators_opt=list(range(10,35,5))
oob_score_opt=['True','False']
criterion_opt=['gini','entropy']
param_grid=dict(n_estimators=n_estimators_opt,criterion=criterion_opt,oob_score=oob_score_opt)
randomf = RandomForestClassifier()
grid=GridSearchCV(randomf,param_grid,cv=10,scoring='log_loss')
grid.fit(X,y)
grid.grid_scores_

print (grid.best_score_)
print (grid.best_params_)
print (grid.best_estimator_)

In [ ]:
forest = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=30, n_jobs=1,
            oob_score='False', random_state=None, verbose=0,
            warm_start=False)
forest.fit(X, y)
importances = forest.feature_importances_
indices = np.argsort(importances)[::-1]


print "Features sorted by their score:"
print sorted(zip(map(lambda x: round(x, 4), forest.feature_importances_), X.columns), 
             reverse=True)

# Print the feature ranking
#print("Feature ranking:")

#for f in range(X.shape[1]):
#    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))   
# Plot the feature importances of the forest
#plt.figure()
#plt.title("Feature importances")
#plt.bar(range(X.shape[1]), importances[indices],
#       color="r", align="center")
#plt.xticks(range(X.shape[1]), indices)
#plt.xlim([-1, X.shape[1]])
#plt.show()

In [ ]:
#Test with GradienBoosting
from sklearn.ensemble import GradientBoostingClassifier
n_estimators_opt=[20,50,100,500]
depth_opt=list(range(3,6))
param_grid=dict(n_estimators=n_estimators_opt,max_depth=depth_opt)
clfgrad = GradientBoostingClassifier(random_state=42)
grid=GridSearchCV(clfgrad,param_grid,cv=5,scoring='log_loss')
grid.fit(X,y)
grid.grid_scores_


print (grid.best_score_)
print (grid.best_params_)
print (grid.best_estimator_)

In [ ]:
#adaboost

from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
n_estimators_opt=list(range(60,120,20))
param_grid=dict(n_estimators=n_estimators_opt)
clfada = AdaBoostClassifier(random_state=42)
grid=GridSearchCV(clfada,param_grid,cv=5,scoring='log_loss')
grid.fit(X,y)
grid.grid_scores_


print (grid.best_score_)
print (grid.best_params_)
print (grid.best_estimator_)

In [ ]:
import xgboost as xgb
from sklearn.grid_search import GridSearchCV
cv_params = {'max_depth': [7], 'min_child_weight': [1],'gamma': [0.75],'learning_rate': [0.05], 'subsample': [0.8],'n_estimators': [200,500]}
ind_params = {'seed':0, 'colsample_bytree': 0.8, 
             'objective': 'binary:logistic','eval_metric': 'logloss'}
xgb_test=xgb.XGBClassifier((ind_params))
#xgb_test=xgb.XGBClassifier()
#grid=GridSearchCV(xgb_test,cv_params,cv=5,scoring='log_loss')
grid=GridSearchCV(xgb_test,cv_params,cv=5,scoring='log_loss')
grid.fit(X,y)
grid.grid_scores_


print (grid.best_score_)
print (grid.best_params_)
print (grid.best_estimator_)


In [ ]:
from sklearn import metrics
xgb_test=xgb.XGBClassifier(objective= 'binary:logistic')
scores = cross_val_score(xgb_test, X, y, cv=5,scoring='log_loss')
scores

In [5]:
prediction=gsearch2.predict_proba(X_test)
submission = pd.DataFrame()
submission["shot_id"] = test_df.shot_id
submission["shot_made_flag"]= prediction[:,0]
submission.to_csv("sub.csv",index=False)

In [ ]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

xgb1 = XGBClassifier(learning_rate =0.1,
 n_estimators=500,max_depth=5,
 min_child_weight=1,gamma=0,
 subsample=0.8,colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,scale_pos_weight=1,seed=27)

xgb_param = xgb1.get_xgb_params()
xgtrain = xgb.DMatrix(X, label=y)
cvresult = xgb.cv(xgb_param,xgtrain, num_boost_round=50, nfold=5,
            metrics='logloss', early_stopping_rounds=50)
print ("log loss score: {}".format(cvresult['test-logloss-mean'].iloc[-1:]))

In [ ]:
xgb1.fit(X,y)
feat_imp = pd.Series(xgb1.booster().get_fscore()).sort_values(ascending=False)
feat_imp.plot(kind='bar', title='Feature Importances')
plt.ylabel('Feature Importance Score')

In [ ]:
#Import libraries:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

train = X

In [ ]:
from sklearn import cross_validation, metrics   #Additional scklearn functions
def modelfit(alg, dtrain_x,dtrain_y, useTrainCV=True, cv_folds=5, early_stopping_rounds=50):
    
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain_x, label=dtrain_y)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=50, nfold=cv_folds,
            metrics='logloss', early_stopping_rounds=early_stopping_rounds)
        alg.set_params(n_estimators=500)
    
    #Fit the algorithm on the data
    alg.fit(dtrain_x, dtrain_y,eval_metric='logloss')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain_x)
    dtrain_predprob = alg.predict_proba(dtrain_x)[:,1]
        
    #Print model report:
    print ("\nModel Report")
    print ("Accuracy (log loss) : %.4g" % metrics.log_loss(dtrain_y, dtrain_predprob))
    #print ("AUC Score (accuracy): %f" % metrics.accuracy_score(dtrain_y, dtrain_predprob))
                    
    feat_imp = pd.Series(alg.booster().get_fscore()).sort_values(ascending=False)
    feat_imp.plot(kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')

In [ ]:
xgb1 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=500,
 max_depth=3,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=27)
modelfit(xgb1, X, y)

In [ ]:
from sklearn.grid_search import GridSearchCV 
param_test1 = {
 'max_depth':list(range(2,8,2)),
 'min_child_weight':list(range(2,8,2))
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=200, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='log_loss',n_jobs=5,iid=False, cv=5)
gsearch1.fit(X,y)
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

In [4]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.grid_search import GridSearchCV 
param_test2 = {
 'n_estimators':[500,550,600,650],
 'max_depth':list(range(5,10,1)),
'gamma':[0,0.5]
}
gsearch2 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=500, max_depth=7,
 min_child_weight=4, gamma=0.5,subsample=1, colsample_bytree=0.55,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test2, scoring='log_loss',n_jobs=5,iid=False, cv=5)
gsearch2.fit(X,y)
gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_

([mean: -0.71055, std: 0.09154, params: {'gamma': 0, 'n_estimators': 500, 'max_depth': 5},
  mean: -0.71460, std: 0.09438, params: {'gamma': 0, 'n_estimators': 550, 'max_depth': 5},
  mean: -0.71888, std: 0.09751, params: {'gamma': 0, 'n_estimators': 600, 'max_depth': 5},
  mean: -0.72282, std: 0.09715, params: {'gamma': 0, 'n_estimators': 650, 'max_depth': 5},
  mean: -0.73089, std: 0.09104, params: {'gamma': 0, 'n_estimators': 500, 'max_depth': 6},
  mean: -0.73417, std: 0.09151, params: {'gamma': 0, 'n_estimators': 550, 'max_depth': 6},
  mean: -0.73878, std: 0.09293, params: {'gamma': 0, 'n_estimators': 600, 'max_depth': 6},
  mean: -0.74246, std: 0.09215, params: {'gamma': 0, 'n_estimators': 650, 'max_depth': 6},
  mean: -0.74214, std: 0.09890, params: {'gamma': 0, 'n_estimators': 500, 'max_depth': 7},
  mean: -0.74806, std: 0.10031, params: {'gamma': 0, 'n_estimators': 550, 'max_depth': 7},
  mean: -0.75407, std: 0.10134, params: {'gamma': 0, 'n_estimators': 600, 'max_depth': 7},

In [7]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

xgb1 = XGBClassifier(learning_rate =0.1,
 n_estimators=500,max_depth=5,
 min_child_weight=1,gamma=0.5,
 subsample=0.8,colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,scale_pos_weight=1,seed=27)

xgb_param = xgb1.get_xgb_params()
xgtrain = xgb.DMatrix(X, label=y)
cvresult = xgb.cv(xgb_param,xgtrain, num_boost_round=50, nfold=5,
            metrics='logloss', early_stopping_rounds=50)
print ("log loss score: {}".format(cvresult['test-logloss-mean'].iloc[-1:]))

log loss score: 49    0.602802
Name: test-logloss-mean, dtype: float64


In [9]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

xgb1 = XGBClassifier(learning_rate =0.1,
 n_estimators=500,max_depth=7,
 min_child_weight=1,gamma=0.5,
 subsample=0.8,colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,scale_pos_weight=1,seed=27)

xgb_param = xgb1.get_xgb_params()
xgtrain = xgb.DMatrix(X, label=y)
cvresult = xgb.cv(xgb_param,xgtrain, num_boost_round=50, nfold=5,
            metrics='logloss', early_stopping_rounds=50)
print ("log loss score: {}".format(cvresult['test-logloss-mean'].iloc[-1:]))

log loss score: 49    0.602048
Name: test-logloss-mean, dtype: float64


In [55]:
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

xgb1 = XGBClassifier(learning_rate =0.01,
 n_estimators=500,max_depth=6,
 min_child_weight=1,gamma=0.6,
 subsample=0.8,colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,scale_pos_weight=1,seed=27)

xgb_param = xgb1.get_xgb_params()
xgtrain = xgb.DMatrix(X, label=y)
cvresult = xgb.cv(xgb_param,xgtrain, num_boost_round=400, nfold=5,
            metrics='logloss', early_stopping_rounds=50)
print ("log loss score: {}".format(cvresult['test-logloss-mean'].iloc[-1:]))

log loss score: 399    0.601774
Name: test-logloss-mean, dtype: float64


In [56]:
cvresult

,test-logloss-mean,test-logloss-std,train-logloss-mean,train-logloss-std
0,0.691597,0.000043,0.691528,0.000016
1,0.690074,0.000105,0.689924,0.000020
2,0.688637,0.000113,0.688408,0.000086
3,0.687222,0.000275,0.686922,0.000136
4,0.685821,0.000344,0.685447,0.000185
5,0.684412,0.000395,0.683963,0.000194
6,0.683082,0.000379,0.682550,0.000240
7,0.681727,0.000391,0.681105,0.000256
8,0.680435,0.000439,0.679739,0.000277
9,0.679111,0.000461,0.678340,0.000296


In [58]:
def fitting(gamm):
    import xgboost as xgb
    from xgboost.sklearn import XGBClassifier
    from sklearn import cross_validation, metrics

    xgb1 = XGBClassifier(learning_rate =0.1,
     n_estimators=500,max_depth=6,
     min_child_weight=1,gamma=gamm,
     subsample=0.8,colsample_bytree=0.8,
     objective= 'binary:logistic',
     nthread=4,scale_pos_weight=1,seed=27)

    xgb_param = xgb1.get_xgb_params()
    xgtrain = xgb.DMatrix(X, label=y)
    cvresult = xgb.cv(xgb_param,xgtrain, num_boost_round=50, nfold=5,
            metrics='logloss', early_stopping_rounds=50)
    return (cvresult['test-logloss-mean'][49])

In [66]:
n_estimator=[400,450,500,550,600]
max_depth=[4,5,6,7,8,9]
gamm=[x / 100 for x in list(range(50,70,2))]
min_c=[1,2,3,4,5,6,7]
learn=[0.1,0.05,0.02,0.01]
score=[]
for n in gamm:
    score.append(fitting(n))
print (score)

[0.60204360000000001, 0.60197319999999999, 0.60205580000000003, 0.6020629999999999, 0.60199419999999992, 0.60188780000000008, 0.60214299999999998, 0.60208939999999989, 0.60199999999999998, 0.60192799999999991]


In [57]:
xgb1.bst = xgb.train(xgb_param, xgtrain, num_boost_round=400)
dtest = xgb.DMatrix(X_test)
test_y = xgb1.bst.predict(dtest)

submission = pd.DataFrame()
submission["shot_id"] = test_df.shot_id
submission["shot_made_flag"]= test_y
submission.to_csv("sub.csv",index=False)